In [1]:
# Exactly reproducing results of tsc bakeoff paper 2023

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
import time
from tsml_eval.evaluation.storage import load_classifier_results
from tsml_eval.publications.y2023.tsc_bakeoff.run_experiments import _run_experiment

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
bakeoff_classifier_list = [
    # distance based
    "KNeighborsTimeSeriesClassifier","GRAILClassifier",
    # feature based
    "Catch22Classifier","FreshPRINCEClassifier",
    # shapelet based
    "RDSTClassifier","RandomShapeletForestClassifier", 
    # interval based
    "RSTSFClassifier","RandomIntervalSpectralEnsembleClassifier","TimeSeriesForestClassifier",
    # dictionary based
    "BOSSEnsemble","weasel v1","weasel_v2",
    # convolution based
    "RocketClassifier","Arsenal",
    # deep learning
    "CNNClassifier","ResNetClassifier",
    # hybrid
    ]

In [2]:

data_path="112UCRFolds"
results_path="reprod_results/"

#classifier listed in categories
distance_based = ["1NN-DTW", "GRAIL"] #PF and EE are best
feature_based = ["Catch22", "FreshPRINCE", "TSFresh", "Signatures"] #FreshPRINCE and TSFrech are best
shapelet_based = ["STC", "RDST", "RSF", "MrSQM"]  #RDST and MrSQM are best
interval_based = ["R-STSF", "RISE", "TSF", "CIF", "STSF", "DrCIF", "QUANT"] #QUANT and DrCIF are best (also QUANT is fast)
dictionary_based = ["BOSS", "cBOSS", "TDE", "WEASEL", "WEASEL-D"] #WEASELv2 and TDE are best
convolution_based = ["ROCKET","MiniROCKET","MultiROCKET","Arsenal","Hydra","Hydra-MultiROCKET"] #MR-Hydra and MR are best ## "multirocket-hydra"
deep_learning = ["CNN", "ResNet", "InceptionTime", "H-InceptionTime", "LITETime"] #CNN is no better than 1NN-DTW, ResNet > CNN, InceptionTime are best
hybrid = ["HC1", "HC2", "RIST"] #HC2 and RIST are best

current_classifiers_beef = ["1NN-DTW", "GRAIL","Catch22", "FreshPRINCE","RDST", "RSF","R-STSF",
                        "RISE", "TSF","BOSS","WEASEL","ROCKET","Arsenal","CNN", "ResNet",
                         "HC2", "H-InceptionTime", "MR-Hydra" ]
current_classifiers = ["1NN-DTW", "GRAIL","Catch22", "FreshPRINCE","RDST", "RSF","R-STSF",
                        "RISE", "TSF","BOSS","WEASEL","ROCKET","Arsenal","CNN", "ResNet", "MR-Hydra"]

current_dataset_names = ["Chinatown"]

classifier="ROCKET" #"TSFresh" #"Hydra"
classifier="1NN-DTW"
 
#dataset_name="ArrowHead"
#dataset_name="Fish"
#dataset_name="Chinatown"
 
resample_id=str(0)
predefined_resample=True

def run_multiple_experiments(current_classifiers, current_ds_n, res_id, predefined_resample=True):
    """RUN multiple classifier over multiple datasets and write the result into
    '/reprod_results/{current_classifier}/Predictions/{current_dataset_name}/testResample0.csv'
    using the default resample id = 0 """
    #TODO store the train&predict time for the classifier&dataset pair in a csv. maybe in the path above?
    for cl_ in current_classifiers:
        classifier= cl_
        for ds_ in current_ds_n:
            dataset_name=ds_
            start_time = time.time()
            _run_experiment(
                args=[
                    data_path,
                    results_path,
                    classifier,
                    dataset_name,
                    res_id,
                ],
                predefined_resample = predefined_resample,
            )
            train_time = time.time() - start_time
            print("---------------------------- "+ f"Train time = {train_time:.2f}s")

run_multiple_experiments(current_classifiers=current_classifiers, current_ds_n=current_dataset_names, res_id=resample_id)

Input args =  ['112UCRFolds', 'reprod_results/', '1NN-DTW', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results/', 'GRAIL', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results/', 'Catch22', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results/', 'FreshPRINCE', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results/', 'RDST', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results/', 'RSF', 'Chinatown', '0']
Ignoring, results already present
---------------------------- Train time = 0.00s
Input args =  ['112UCRFolds', 'reprod_results

In [42]:
print("ds_Beef_Comparison")
print("algorithm    ||    current     ||      benchmark")

ds_Beef_Comparison
algorithm    ||    current     ||      benchmark


In [ ]:
import numpy as np
import pandas as pd


def compare_acc_(current_clfs, current_ds="Beef", print_res=False):
    """
    RECEIVE: current_clfs: list of classifiers which should be investigated AND results are present in 
                           directory_current. NOTE results must be present in the directory format:
                           "reprod_results/<classifier_name>/Predictions/<dataset_name>/testResample0.csv"
                           benchmark results need to be accesible via:
                           "benchmark/<classifier_name>/_accuracy.csv"
             current_ds: name of dataset which where the results should be compared
             print_res: option for printing if no df_object is created.
    RETURN: comparison df with format:       algorithm  ||  current_acc  ||  benchmark_acc  ||  deviation  
    """
    dataset_= current_ds
    directory_benchmark = "benchmark/"
    directory_current = "reprod_results/"
    comparison_df = pd.DataFrame(columns=["algorithm", "current_acc", "benchmark_acc", "deviation"])
    comparison_df = comparison_df.astype({"algorithm": "string",
                                          "current_acc": "float64",
                                          "benchmark_acc": "float64",
                                          "deviation": "float64"})

    for cl_ in current_clfs:
        filename_current = cl_ + "/Predictions/" + dataset_ + "/testResample0.csv"
        filename_benchmark = cl_ + "_accuracy.csv"
        file_path_benchmark = os.path.join(directory_benchmark, filename_benchmark)
        file_path_current = os.path.join(directory_current,  filename_current)
        df_benchmark = pd.read_csv(file_path_benchmark)
        #df_current = pd.read_csv(file_path_current)
        #print(file_path_current)
        current_results = load_classifier_results(file_path_current)
            
        if not df_benchmark.empty:  # Check if the CSV is not empty
            benchmark_acc = df_benchmark.loc[df_benchmark["Resamples:"]==dataset_, "0"].values
        else:
            print("The CSV file is empty!")    

        current_acc = current_results.accuracy

        curr_acc = np.round(current_acc,6)
        dev = np.round(np.abs(curr_acc -benchmark_acc),6)
        new_row1 = pd.DataFrame({'algorithm': [cl_], 'current_acc': [curr_acc], 'benchmark_acc':benchmark_acc, "deviation": dev})
        comparison_df = pd.concat([comparison_df, new_row1], ignore_index=True) # Appending 

    if print_res==True:
        print(comparison_df)

    return(comparison_df)

#comparison_Beef = compare_acc_(current_clfs=current_classifiers, current_ds="Beef")
comparison_ArrowHead = compare_acc_(current_clfs=current_classifiers, current_ds="ArrowHead")

In [271]:
comparison_ArrowHead

,algorithm,current_acc,benchmark_acc,deviation
0,1NN-DTW,0.702857,0.702857,0.000000
1,GRAIL,0.828571,0.794286,0.034285
2,Catch22,0.725714,0.725714,0.000000
3,FreshPRINCE,0.600000,0.628571,0.028571
4,RDST,0.862857,0.857143,0.005714
5,RSF,0.691429,0.697143,0.005714
6,R-STSF,0.754286,0.788571,0.034285
7,RISE,0.800000,0.788571,0.011429
8,TSF,0.708571,0.731429,0.022858
9,BOSS,0.834286,0.834286,0.000000


In [275]:
comparison_Chinatown = compare_acc_(current_clfs=current_classifiers, current_ds="Chinatown")
comparison_Chinatown


,algorithm,current_acc,benchmark_acc,deviation
0,1NN-DTW,0.973761,0.973761,0.000000
1,GRAIL,0.274052,0.274052,0.000000
2,Catch22,0.915452,0.915452,0.000000
3,FreshPRINCE,0.973761,0.982507,0.008746
4,RDST,0.976676,0.976676,0.000000
5,RSF,0.976676,0.950437,0.026239
6,R-STSF,0.979592,0.979592,0.000000
7,RISE,0.959184,0.970845,0.011661
8,TSF,0.976676,0.976676,0.000000
9,BOSS,0.895044,0.895044,0.000000


In [267]:
comparison_Beef= compare_acc_(current_clfs=current_classifiers_beef, current_ds="Beef")
comparison_Beef

,algorithm,current_acc,benchmark_acc,deviation
0,1NN-DTW,0.633333,0.633333,0.000000
1,GRAIL,0.500000,0.566667,0.066667
2,Catch22,0.600000,0.600000,0.000000
3,FreshPRINCE,0.833333,0.800000,0.033333
4,RDST,0.866667,0.833333,0.033334
5,RSF,0.533333,0.566667,0.033334
6,R-STSF,0.766667,0.866667,0.100000
7,RISE,0.833333,0.866667,0.033334
8,TSF,0.800000,0.800000,0.000000
9,BOSS,0.766667,0.766667,0.000000


In [272]:
comparison_Fish= compare_acc_(current_clfs=current_classifiers, current_ds="Fish")
comparison_Fish

,algorithm,current_acc,benchmark_acc,deviation
0,1NN-DTW,0.822857,0.822857,0.000000
1,GRAIL,0.880000,0.880000,0.000000
2,Catch22,0.794286,0.794286,0.000000
3,FreshPRINCE,0.925714,0.914286,0.011428
4,RDST,0.994286,0.994286,0.000000
5,RSF,0.891429,0.862857,0.028572
6,R-STSF,0.920000,0.925714,0.005714
7,RISE,0.868571,0.862857,0.005714
8,TSF,0.817143,0.800000,0.017143
9,BOSS,0.977143,0.977143,0.000000


In [237]:
ds_ = "ArrowHead"

df_current = pd.read_csv("reprod_results/RDST/Predictions/"+ ds_ +"/testResample0.csv")
if np.isnan(float(df_current[ds_][1])) or float(df_current[ds_][1]) == -1:
    print(df_current[ds_].keys()[1][0])

df_current[ds_][1]
    

'1103'

In [78]:
df_benchmark.loc[df["Resamples:"]==dataset_, "0":"4"]

,0,1,2,3,4
4,0.633333,0.533333,0.533333,0.5,0.566667


In [50]:
df_current["Beef"][1]

'0.6333333333333333'